In [1]:
def pendent_pair(Vprime, V, S, f, data):
    """ return Queyranne's pendent pair for function f over finite set V,
        where the first (arbitary) input element x is V[0] """
    x = 0
    vnew = Vprime[x]
    n = len(Vprime)
    Wi = []
    used = np.zeros((n,1))
    used[x] = 1

    for i in range(n-1):
        vold = vnew
        Wi = Wi + S[vold]

        ## update keys
        keys = np.ones((n,1))*np.inf
        for j in range(n):
            if used[j]:
                continue
            keys[j] = f(Wi + S[Vprime[j]], V, data) - f(S[Vprime[j]], V, data)

        ## extract min
        argmin = np.argmin(keys)
        vnew = Vprime[argmin]
        used[argmin] = 1
        fval = np.min(keys)

    s = vold
    t = vnew

    return s, t, fval


def diff(A, B):
    # Check if B is an integer
    if isinstance(A, int):
        set_A = {A}  # Convert the integer to a set with a single element
    else:
        set_A = set(A)  # B is already a set
    
    
    # Check if B is an integer
    if isinstance(B, int):
        set_B = {B}  # Convert the integer to a set with a single element
    else:
        set_B = set(B)  # B is already a set
        
        
    result = [x for x in set_A if x not in set_B]
    return result


def optimal_set(V, f, data):
    """ implement Queyranne's algorithm for finding the minimum of symmetric
        submodular function f over the finite set V
        to utilize: define a function f(S, V, params). Queyranne's algorithm will
        minimize f[S] + f[V\S] given any necessary parameters in params """

    n = len(V)
    S = [[] for _ in range(n)]
    for i in range(n):
        S[i] = [V[i]]

    p = np.zeros((n-1,1))
    A = []
    idxs = range(n)
    for i in range(n-1):
        ## find a pendant pair
        t, u, fval = pendent_pair(idxs, V, S, f, data)
        ## candidate solution
        A.append(S[u])
        p[i] = f(S[u],V,data)
        S[t] = [*S[t], *S[u]]
        idxs = diff(idxs, u)
        S[u] = []

    ## return minimum solution
    i = np.argmin(p)
    R = A[i]
    fval = p[i]

    ## make R look pretty
    notR = diff(V,R)
    R = sorted(R)
    notR = sorted(notR)

    if R[0] < notR[0]:
        R = (list(R),list(notR))
    else:
        R = (list(notR),list(R))

    return list(R), fval


In [ ]:
# Function to compute the determinant of the covariance matrix for a set of variables
def det_cov(set_variables, all_variables, data):
    df = data
    selected_variables = df[set_variables]
    
    covariance_matrix = selected_variables.cov()
    
    determinant_selected = np.linalg.det(covariance_matrix)
    
    #all_variables = df.columns
    complement_variables = [var for var in all_variables if var not in set_variables]

    complement_selected_variables = df[complement_variables]
    
    covariance_matrix_complement = complement_selected_variables.cov()
    
    determinant_complement = np.linalg.det(covariance_matrix_complement)
    
    return determinant_selected+determinant_complement